In [10]:
### Utility Functions

def data_ingestion():
    
    from IPython.display import display, HTML
    import pandas as pd
    import sqlite3
    from sqlite3 import Error
    import numpy as np
    import datetime

    def create_connection(db_file, delete_db=False):
        import os
        if delete_db and os.path.exists(db_file):
            os.remove(db_file)

        conn = None
        try:
            conn = sqlite3.connect(db_file)
            conn.execute("PRAGMA foreign_keys = 1")
        except Error as e:
            print(e)

        return conn


    def create_table(conn, create_table_sql, drop_table_name=None):

        if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
            try:
                c = conn.cursor()
                c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
            except Error as e:
                print(e)

        try:
            c = conn.cursor()
            c.execute(create_table_sql)
        except Error as e:
            print(e)

    def create_users_table(sql_statement, conn):
        cur = conn.cursor()
        cur.execute(sql_statement)

        rows = cur.fetchall()

        return rows


    #Insert Functions - Using Executemany for Efficient Insertion
    def insert_age_gender_buckets(conn, values):                                       

        cur = conn.cursor()
        cur.executemany(" INSERT INTO age_gender_buckets (country_destination,gender,population_in_thousands,year,age_bucket_min,age_bucket_max) VALUES (?,?,?,?,?,?) ",values )
        return cur.lastrowid

    def insert_sessions(conn, values):                                       

        cur = conn.cursor()
        cur.executemany(" INSERT INTO session (user_id,action,action_type,action_detail,device_type,secs_elaspsed) VALUES (?,?,?,?,?,?) ",values )
        return cur.lastrowid

    def insert_countries(conn, values):                                       

        cur = conn.cursor()
        cur.executemany(" INSERT INTO countries (country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance) VALUES (?,?,?,?,?,?,?) ",values )
        return cur.lastrowid

    def insert_users(conn, values):                                       

        cur = conn.cursor()
        cur.executemany(" INSERT INTO user (user_id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) ",values )
        return cur.lastrowid



    conn = create_connection("airbnb.db",True)
    # Creating and Inserting in Age Gender Buckets Table
    def create_age_gender_buckets_table(conn):    

        age_gender_buckets_sql_statement = """CREATE TABLE age_gender_buckets (                                       
                                                      age_gender_buckets_id INTEGER NOT NULL PRIMARY KEY,
                                                      country_destination TEXT NOT NULL,
                                                      gender TEXT NOT NULL,
                                                      population_in_thousands INTEGER NOT NULL,
                                                      year INTEGER NOT NULL,
                                                      age_bucket_min INTEGER NOT NULL,
                                                      age_bucket_max INTEGER NOT NULL
                                                        );"""
        create_table(conn,age_gender_buckets_sql_statement,"age_gender_buckets")
        insert_age_gender_buckets(conn,age_gender_data)

        conn.commit()
        return conn

    # Creating and Inserting in Sessions Table
    def create_sessions_table(conn):    

        session_sql_statement = """CREATE TABLE session (                                       
                                                      session_id INTEGER NOT NULL PRIMARY KEY,
                                                      user_id TEXT ,
                                                      action TEXT ,
                                                      action_type TEXT ,
                                                      action_detail TEXT ,
                                                      device_type TEXT ,
                                                      secs_elaspsed REAL ,
                                                      FOREIGN KEY(user_id) REFERENCES user(user_id)
                                                        );"""
        create_table(conn,session_sql_statement,"session")
        insert_sessions(conn,sessions_data)

        conn.commit()
        return conn

    # Creating and Inserting in Countries Table
    def create_countries_table(conn):    

        country_des_sql_statement = """CREATE TABLE countries (                                       
                                                      country_destination_id INTEGER NOT NULL PRIMARY KEY,
                                                      country_destination TEXT NOT NULL,
                                                      lat_destination REAL NOT NULL,
                                                      lng_destination REAL NOT NULL,
                                                      distance_km REAL NOT NULL,
                                                      destination_km2 REAL NOT NULL,
                                                      destination_language TEXT NOT NULL,
                                                      language_levenshtein_distance REAL NOT NULL
                                                        );"""
        create_table(conn,country_des_sql_statement,"countries")
        insert_countries(conn,country_data)

        conn.commit()
        return conn

    # Creating and Inserting in Users Table
    def create_users_table(conn):    

        user_sql_statement = """CREATE TABLE user (                                       

                                                      user_id TEXT NOT NULL PRIMARY KEY,
                                                      date_account_created TEXT NOT NULL ,
                                                      timestamp_first_active TEXT NOT NULL,
                                                      date_first_booking TEXT,
                                                      gender TEXT NOT NULL,
                                                      age INTEGER ,
                                                      signup_method TEXT NOT NULL,
                                                      signup_flow INTEGER NOT NULL,
                                                      language TEXT NOT NULL,
                                                      affiliate_channel TEXT NOT NULL,
                                                      affiliate_provider TEXT NOT NULL,
                                                      first_affiliate_tracked TEXT ,
                                                      signup_app TEXT NOT NULL,
                                                      first_device_type TEXT NOT NULL,
                                                      first_browser TEXT NOT NULL,
                                                      country_destination NOT NULL
                                                        );"""
        create_table(conn,user_sql_statement,"user")
        insert_users(conn,users_data)

        conn.commit()
        return conn

    # Reading Users csv file and cleaning it before inserting it to sqlite table
    df = pd.read_csv("train_users_2.csv")
    df['timestamp_first_active'] = df['timestamp_first_active'].apply(lambda x: str(datetime.datetime.strptime(str(x), '%Y%m%d%H%M%S')))
    df.signup_flow = df.signup_flow.astype(str)     
    df = df.replace('-unknown-', 'unknown')

    # Reading Age Gender Bucket and Countries csv file and cleaning it before inserting it to sqlite table
    age_gender_buckets = pd.read_csv('age_gender_bkts.csv')
    age_gender_buckets = age_gender_buckets.replace('-unknown-', 'unknown')
    country_df = pd.read_csv('countries.csv')

    # Reading User's sessions csv file and cleaning it before inserting it to sqlite table
    sessions = pd.read_csv('sessions.csv')
    sessions = sessions.replace('-unknown-', 'unknown')
    sessions = sessions.dropna(subset=['user_id'])

    # Normalizing Age Gender Bucket Data
    age_gender_buckets["age_bucket"] = age_gender_buckets.age_bucket.replace("100+","100-200")
    age_gender_buckets["age_min"] =  age_gender_buckets.age_bucket.apply(lambda x: x.split('-')[0] )
    age_gender_buckets["age_max"] = age_gender_buckets.age_bucket.apply(lambda x: x.split('-')[1] )
    age_gender_buckets = age_gender_buckets.drop(columns=["age_bucket"])

    # Creating tuples from dataframe
    df_rec = df.to_records(index=False)
    age_rec = age_gender_buckets.to_records(index=False)
    ses_records = sessions.to_records(index=False)
    count_rec = country_df.to_records(index=False)

    # Creating List of tuples before inserting to sqlite table
    users_data = list(df_rec)
    age_gender_data = list(age_rec)
    country_data = list(count_rec)
    sessions_data = list(ses_records)

    # Creating Age Gender Bucket Table
    create_age_gender_buckets_table(conn)

    # Creating Users Table
    create_users_table(conn)

    df = pd.read_sql_query("Select * from user",conn)
    # Creating a temporary dataframe to remove id's from Sessions table which do not exist in Users Table because those are useless
    temp_df = df[["user_id"]]

    sessions = pd.merge(sessions,temp_df,on="user_id")
    ses_records = sessions.to_records(index=False)
    sessions_data = list(ses_records)

    # Creating Sessions Table
    create_sessions_table(conn)

    # Creating Countries Table
    create_countries_table(conn)
    